<a href="https://colab.research.google.com/github/ANCHARAN/ANCHARAN/blob/main/test_app_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

/bin/bash: line 1: mKdir: command not found


In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform pymupdf google-cloud-storage

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
  File "/usr/local/lib/python3.10/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
# Define project information

import sys

PROJECT_ID = "turnkey-mender-436305-s2"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# if not running on Colab, try to get the PROJECT_ID automatically
if "google.colab" not in sys.modules:
    import subprocess

    PROJECT_ID = subprocess.check_output(
        ["gcloud", "config", "get-value", "project"], text=True
    ).strip()

print(f"Your project ID is: {PROJECT_ID}")

Your project ID is: turnkey-mender-436305-s2


In [ ]:
import sys

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from IPython.display import display
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Image,
)
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import MultiModalEmbeddingModel

In [ ]:
# Instantiate text model with appropriate name and version
text_model = GenerativeModel("gemini-1.0-pro")  # works with text, code

# Multimodal models: Choose based on your performance/cost needs
multimodal_model_15 = GenerativeModel(
    "gemini-1.5-pro"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - complex reasoning

# Multimodal models: Choose based on your performance/cost needs
multimodal_model_15_flash = GenerativeModel(
    "gemini-1.5-flash"
)  # works with text, code, images, video(with or without audio) and audio(mp3) with 1M input context - faster inference

multimodal_model_10 = GenerativeModel(
    "gemini-1.0-pro-vision-001"
)  # works with text, code, video(without audio) and images with 16k input context

# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")

# Load multimodal embedding model from pre-trained source
multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained(
    "multimodalembedding"
)  # works with image, image with caption(~32 words), video, video with caption(~32 words)

In [ ]:
!gsutil -m -q rsync -r gs://github-repo/rag/intro_multimodal_rag/intro_multimodal_rag_v2 .
print("Download completed")


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Download completed


In [ ]:
from multimodal_qa_with_rag_utils import get_document_metadata, set_global_variable

set_global_variable("text_embedding_model", text_embedding_model)
set_global_variable("multimodal_embedding_model", multimodal_embedding_model)

In [ ]:
from multimodal_qa_with_rag_utils import (
    display_images,
    get_answer_from_qa_system,
    get_gemini_response,
    get_similar_image_from_query,
    get_similar_text_from_query,
    print_text_to_image_citation,
    print_text_to_text_citation,
)

In [ ]:
!pip install flask-ngrok
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import os
import json

In [ ]:
#METHOD FOR EXTRACTING TEXT AND IMAGE METADATA
def extract():
    global text_metadata_df, image_metadata_df, matching_results_text, matching_results_image, images_and_descriptions
    print("Removing pre-existing images folder...")
    os.system('rm -rf images/')  # Remove old images directory if exists

    pdf_folder_path = "/content"
    bucket_name = 'test_1_0'

    # Your custom image description prompt
    image_description_prompt = """
You are a technical image analysis expert. You will be provided with various types of images extracted from documents like research papers, technical blogs, and more.
Your task is to generate concise, accurate descriptions of the images without adding any information you are not confident about.
Focus on capturing the key details, trends, or relationships depicted in the image.

Important Guidelines:
* Prioritize accuracy:  If you are uncertain about any detail, state "Unknown" or "Not visible" instead of guessing.
* Avoid hallucinations: Do not add information that is not directly supported by the image.
* Be specific: Use precise language to describe shapes, colors, textures, and any interactions depicted.
* Consider context: If the image is a screenshot or contains text, incorporate that information into your description.
    """

    # Assuming 'get_document_metadata' is your custom function for extracting text and images
    text_metadata_df, image_metadata_df = get_document_metadata(
        multimodal_model_15,
        pdf_folder_path,
        image_save_dir="images",  # Directory where images will be saved
        image_description_prompt=image_description_prompt,
        embedding_size=1408,
        add_sleep_after_document=True,
        sleep_time_after_document=5,
    )

    print("\n\n --- Completed processing. ---")




def process_pdf(query):
    # Ensure query is a string
    if isinstance(query, dict):
        query = query.get("query", "")  # Extract the query if it's a dict

    # Matching user text query with "chunk_embedding" to find relevant chunks.
    matching_results_text = get_similar_text_from_query(
        query,
        text_metadata_df,
        column_name="text_embedding_chunk",
        top_n=3,
        chunk_text=True,
    )

    # Get the relevant text chunks found across documents based on user query
    context = "\n".join([value["chunk_text"] for key, value in matching_results_text.items()])

    matching_results_image = get_similar_image_from_query(
        text_metadata_df,
        image_metadata_df,
        query=query,
        column_name="text_embedding_from_image_description",  # Use image description text embedding
        image_emb=False,  # Use text embedding instead of image embedding
        top_n=5,
        embedding_size=1408,
    )


    # Print the matched text citations
    print_text_to_text_citation(matching_results_text, print_top=True, chunk_text=True)

    # Print the matched image citations
    print_text_to_image_citation(matching_results_image, print_top=True)


    # Markdown(print_text_to_image_citation(matching_results_image, print_top=True))
    print("\n **** Result: ***** \n")


    # Display the top matching image
    display_images(
        [image["img_path"] for image in matching_results_image if isinstance(image, dict) and "img_path" in image],
        resize_ratio=0.3,
    )

    print("\n **** Results: ***** \n")

    instruction = f"""Answer the question and explain results with the given Image:
    Question: {query}
    Image:
    """

    model_input = [instruction]

    # Iterate over available images and add them to model_input
    for i in range(min(2, len(matching_results_image))):  # Ensure we only access available images, max 4
        model_input.extend([
            "Image:",
            matching_results_image[i]["image_object"],
            "Description:",
            matching_results_image[i]["image_description"]
        ])


    images_and_descriptions = []
    for key, image in matching_results_image.items(): # Use .items() to iterate through key-value pairs
      if not isinstance(image, dict):
        print(f"Item is not a dictionary: {image}")
      elif "img_path" not in image:
        print(f"'img_path' key missing: {image}")
      else: # Access 'img_path' only if it's a dictionary and contains the key
        local_image_path = image["img_path"]
        gcs_image_path = f"images/{os.path.basename(local_image_path)}"
        img_url = upload_to_gcs(local_image_path, bucket_name, gcs_image_path)  # Upload to GCS
        print(img_url)
        images_and_descriptions.append({
                "img_url": img_url,
                "description": image["image_description"]
            })



    # Generate Gemini response with streaming output
    res1=get_gemini_response(
            multimodal_model_15,
            model_input=model_input,
            stream=True,
            safety_settings={HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE},
            generation_config=GenerationConfig(temperature=1, max_output_tokens=8192),
        )

    print_text_to_image_citation(matching_results_image, print_top=True)

    print(res1)


    # Generate final prompt with text and image results
    prompt = f"""Answer the question with the given context. If the specific answer is not in the context, please answer 'I don't know'.
    Question: {query}
    Context: {context + res1}
    Answer:
    """

    # Generate response with Gemini 1.5 Pro based on combined text and image results
    result = get_gemini_response(
        multimodal_model_15_flash,
        model_input=prompt,
        stream=True,
        safety_settings={HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE},
        generation_config=GenerationConfig(temperature=1)
    )

    print(result)

    # return result  # Return the generated result

    return {
        "result": result,
        "images_and_descriptions": images_and_descriptions
    }

In [ ]:
# Install ngrok
!pip install pyngrok
!ngrok config add-authtoken 2mjuubaIHrkvbM9ZnOfwLFdL02H_3n5EYsCJZh8PQ6R51zZ8U  # Replace with your ngrok auth token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok flask

from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import os

# Initialize Flask app
app = Flask(__name__)

# Start ngrok tunnel
public_url = ngrok.connect(addr='5000')
print("Ngrok tunnel URL:", public_url)

# Define the HTML template
html_template_v8 = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Multimodal RAG System</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css">
    <style>
        .loading { animation: spin 1s linear infinite; }
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
        .scroll-area { max-height: 300px; overflow-y: auto; }
        .image-container { margin-bottom: 1.5rem; }
        .image-description { margin-top: 0.5rem; text-align: center; color: #4B5563; }
        .gallery-overlay { position: fixed; top: 0; left: 0; right: 0; bottom: 0; background-color: rgba(0, 0, 0, 0.8); z-index: 50; display: none; justify-content: center; align-items: center; padding: 1rem; }
        .gallery-img { max-height: 70vh; object-fit: contain; border-radius: 0.5rem; }
    </style>
</head>
<body class="bg-gray-100">
    <div class="container mx-auto p-4 max-w-4xl">
        <h1 class="text-3xl font-bold mb-8 text-center">Multimodal RAG System</h1>

        <div class="bg-white shadow-md rounded-lg mb-8">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Upload File</h2>
                <form id="uploadForm" class="space-y-4">
                    <input type="file" name="file" id="fileInput" required class="border p-2 rounded w-full" />
                    <button type="submit" id="uploadButton" class="bg-blue-500 text-white p-2 rounded w-full">
                        <span id="uploadButtonText">Upload</span>
                    </button>
                </form>
                <p id="uploadMessage" class="mt-2 text-sm text-gray-600"></p>
            </div>
        </div>

        <div class="bg-white shadow-md rounded-lg mb-8">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Process Query</h2>
                <form id="queryForm" class="space-y-4">
                    <input type="text" name="query" id="queryInput" placeholder="Enter your query" required class="border p-2 rounded w-full" />
                    <button type="submit" id="processButton" class="bg-green-500 text-white p-2 rounded w-full">
                        <span id="processButtonText">Process Query</span>
                    </button>
                </form>
            </div>
        </div>

        <div id="resultContainer" class="hidden bg-white shadow-md rounded-lg">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Query Results</h2>
                <p id="queryResultMessage" class="mb-4"></p>
                <div class="scroll-area rounded-md border p-4">
                    <p id="queryResultText" class="text-gray-700 leading-relaxed"></p>
                </div>

                <h3 class="text-lg font-semibold mb-4">Images and Descriptions</h3>
                <div id="imagesContainer" class="grid grid-cols-2 sm:grid-cols-3 md:grid-cols-4 gap-4"></div>
            </div>
        </div>
    </div>

    <!-- Image Gallery Overlay -->
    <div id="galleryOverlay" class="gallery-overlay">
        <div class="relative w-full max-w-4xl">
            <button id="closeGallery" class="absolute top-2 right-2 text-white z-10">
                <i class="fas fa-times h-6 w-6"></i>
            </button>
            <div class="flex items-center justify-between">
                <button id="prevImage" class="text-white">
                    <i class="fas fa-chevron-left h-8 w-8"></i>
                </button>
                <img id="selectedImage" class="gallery-img" src="" alt="Selected" />
                <button id="nextImage" class="text-white">
                    <i class="fas fa-chevron-right h-8 w-8"></i>
                </button>
            </div>
            <div class="scroll-area mt-4 h-24 rounded-md bg-white/10 p-4">
                <p id="imageDescription" class="text-white text-sm leading-relaxed"></p>
            </div>
        </div>
    </div>

    <script>
        let images = []; // Store images globally
        document.getElementById('uploadForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const uploadButton = document.getElementById('uploadButton');
            const uploadButtonText = document.getElementById('uploadButtonText');
            const uploadMessage = document.getElementById('uploadMessage');

            uploadButton.disabled = true;
            uploadButtonText.innerHTML = '<i class="fas fa-spinner fa-spin loading"></i> Uploading...';

            const formData = new FormData(event.target);
            try {
                const response = await fetch('/upload', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                uploadMessage.innerText = result.message;
                document.getElementById('processButton').disabled = false;
            } catch (error) {
                console.error('Error uploading file:', error);
                uploadMessage.innerText = 'Error uploading file';
            } finally {
                uploadButton.disabled = false;
                uploadButtonText.innerHTML = 'Upload';
            }
        });

        document.getElementById('queryForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const processButton = document.getElementById('processButton');
            const processButtonText = document.getElementById('processButtonText');
            const queryResultMessage = document.getElementById('queryResultMessage');
            const imagesContainer = document.getElementById('imagesContainer');
            const resultContainer = document.getElementById('resultContainer');
            const queryResultText = document.getElementById('queryResultText');

            processButton.disabled = true;
            processButtonText.innerHTML = '<i class="fas fa-spinner fa-spin loading"></i> Processing...';

            const formData = new FormData(event.target);
            try {
                const response = await fetch('/process-query', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                if (result.error) {
                    queryResultMessage.innerText = result.error;
                    imagesContainer.innerHTML = '';
                } else {
                    queryResultMessage.innerText = result.result;
                    queryResultText.innerText = result.result; // Set text result
                    imagesContainer.innerHTML = '';
                    images = result.images_and_descriptions; // Store image data globally
                    images.forEach((image, index) => {
                        const imageDiv = document.createElement('div');
                        imageDiv.className = 'image-container';
                        imageDiv.innerHTML = `
                            <img src="${image.img_url}" alt="Thumbnail ${index + 1}" class="max-w-full h-auto rounded-lg shadow-md cursor-pointer" onclick="openImage(${index})" />
                            <p class="image-description">${image.description}</p>
                        `;
                        imagesContainer.appendChild(imageDiv);
                    });
                }
                resultContainer.classList.remove('hidden');
            } catch (error) {
                console.error('Error processing query:', error);
                queryResultMessage.innerText = 'An error occurred while processing the query.';
                imagesContainer.innerHTML = '';
            } finally {
                processButton.disabled = false;
                processButtonText.innerHTML = 'Process Query';
            }
        });

        // Image gallery functions
        let selectedIndex = null;

        function openImage(index) {
            selectedIndex = index;
            document.getElementById('galleryOverlay').style.display = 'flex';
            displayImage();
        }

        function closeImage() {
            document.getElementById('galleryOverlay').style.display = 'none';
        }

        function nextImage() {
            selectedIndex = (selectedIndex === null || selectedIndex === images.length - 1) ? 0 : selectedIndex + 1;
            displayImage();
        }

        function prevImage() {
            selectedIndex = (selectedIndex === null || selectedIndex === 0) ? images.length - 1 : selectedIndex - 1;
            displayImage();
        }

        function displayImage() {
            const selectedImage = document.getElementById('selectedImage');
            const imageDescription = document.getElementById('imageDescription');
            selectedImage.src = images[selectedIndex].img_url;
            imageDescription.innerText = images[selectedIndex].description;
        }

        // Attach click events for gallery navigation
        document.getElementById('closeGallery').addEventListener('click', closeImage);
        document.getElementById('nextImage').addEventListener('click', nextImage);
        document.getElementById('prevImage').addEventListener('click', prevImage);
    </script>
</body>
</html>
"""

html_template_v9="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Multimodal RAG System</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css">
    <style>
        .loading {
            animation: spin 1s linear infinite;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .scroll-area {
            max-height: 300px;
            overflow-y: auto;
        }
        .image-container {
            margin-bottom: 1.5rem; /* Margin between each image and description */
        }
        .image-description {
            margin-top: 0.5rem; /* Margin between image and description */
            padding: 1rem;
            background-color: #F3F4F6; /* Light gray background for the description box */
            border-radius: 0.5rem; /* Rounded corners */
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); /* Subtle shadow */
            display: flex;
            flex-direction: column;
            align-items: center;
            position: relative;
        }
        .more-link {
            margin-top: 0.5rem; /* Space above the "more" link */
            color: #3B82F6; /* Tailwind blue for the link */
            cursor: pointer;
            text-decoration: underline;
        }
        .gallery-overlay {
            position: fixed;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background-color: rgba(0, 0, 0, 0.8);
            z-index: 50;
            display: none;
            justify-content: center;
            align-items: center;
            padding: 1rem;
        }
        .gallery-img {
            max-height: 70vh;
            object-fit: contain;
            border-radius: 0.5rem; /* Rounded corners */
        }
    </style>
</head>
<body class="bg-gray-100">
    <div class="container mx-auto p-4 max-w-4xl">
        <h1 class="text-3xl font-bold mb-8 text-center">Multimodal RAG System</h1>

        <div class="bg-white shadow-md rounded-lg mb-8">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Upload File</h2>
                <form id="uploadForm" class="space-y-4">
                    <input type="file" name="file" id="fileInput" required class="border p-2 rounded w-full" />
                    <button type="submit" id="uploadButton" class="bg-blue-500 text-white p-2 rounded w-full">
                        <span id="uploadButtonText">Upload</span>
                    </button>
                </form>
                <p id="uploadMessage" class="mt-2 text-sm text-gray-600"></p>
            </div>
        </div>

        <div class="bg-white shadow-md rounded-lg mb-8">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Process Query</h2>
                <form id="queryForm" class="space-y-4">
                    <input
                        type="text"
                        name="query"
                        id="queryInput"
                        placeholder="Enter your query"
                        required
                        class="border p-2 rounded w-full"
                    />
                    <button type="submit" id="processButton" class="bg-green-500 text-white p-2 rounded w-full">
                        <span id="processButtonText">Process Query</span>
                    </button>
                </form>
            </div>
        </div>

        <div id="resultContainer" class="hidden bg-white shadow-md rounded-lg">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Query Results</h2>
                <p id="queryResultMessage" class="mb-4 text-center"></p>
                <div class="scroll-area rounded-md border p-4">
                    <p id="queryResultText" class="text-gray-700 leading-relaxed text-center"></p>
                </div>

                <h3 class="text-lg font-semibold mb-4">Images and Descriptions</h3>
                <div id="imagesContainer" class="flex flex-col gap-4"></div>
            </div>
        </div>
    </div>

    <!-- Image Gallery Overlay -->
    <div id="galleryOverlay" class="gallery-overlay">
        <div class="relative w-full max-w-4xl">
            <button id="closeGallery" class="absolute top-2 right-2 text-white z-10">
                <i class="fas fa-times h-6 w-6"></i>
            </button>
            <div class="flex items-center justify-between">
                <button id="prevImage" class="text-white">
                    <i class="fas fa-chevron-left h-8 w-8"></i>
                </button>
                <img id="selectedImage" class="gallery-img" src="" alt="Selected" />
                <button id="nextImage" class="text-white">
                    <i class="fas fa-chevron-right h-8 w-8"></i>
                </button>
            </div>
            <div class="scroll-area mt-4 h-24 rounded-md bg-white/10 p-4">
                <p id="imageDescription" class="text-white text-sm leading-relaxed"></p>
            </div>
        </div>
    </div>

    <script>
        document.getElementById('uploadForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const uploadButton = document.getElementById('uploadButton');
            const uploadButtonText = document.getElementById('uploadButtonText');
            const uploadMessage = document.getElementById('uploadMessage');

            uploadButton.disabled = true;
            uploadButtonText.innerHTML = '<i class="fas fa-spinner fa-spin loading"></i> Uploading...';

            const formData = new FormData(event.target);
            try {
                const response = await fetch('/upload', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                uploadMessage.innerText = result.message;
                document.getElementById('processButton').disabled = false;
            } catch (error) {
                console.error('Error uploading file:', error);
                uploadMessage.innerText = 'Error uploading file';
            } finally {
                uploadButton.disabled = false;
                uploadButtonText.innerHTML = 'Upload';
            }
        });

        document.getElementById('queryForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const processButton = document.getElementById('processButton');
            const processButtonText = document.getElementById('processButtonText');
            const queryResultMessage = document.getElementById('queryResultMessage');
            const imagesContainer = document.getElementById('imagesContainer');
            const resultContainer = document.getElementById('resultContainer');
            const queryResultText = document.getElementById('queryResultText');

            processButton.disabled = true;
            processButtonText.innerHTML = '<i class="fas fa-spinner fa-spin loading"></i> Processing...';

            const formData = new FormData(event.target);
            try {
                const response = await fetch('/process-query', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                if (result.error) {
                    queryResultMessage.innerText = result.error;
                    imagesContainer.innerHTML = '';
                } else {
                    queryResultMessage.innerText = result.result;
                    queryResultText.innerText = result.result; // Set text result
                    imagesContainer.innerHTML = '';
                    result.images_and_descriptions.forEach((image, index) => {
                        const imageDiv = document.createElement('div');
                        imageDiv.className = 'image-container';
                        imageDiv.innerHTML = `
                            <img src="${image.img_url}" alt="Image ${index + 1}" class="max-w-full h-auto rounded-lg shadow-md" />
                            <div class="image-description">
                                <p>${image.description.slice(0, 50)}...</p>
                                <span class="more-link" onclick="showMore('${image.description}')">more</span>
                            </div>
                        `;
                        imagesContainer.appendChild(imageDiv);
                    });
                }
                resultContainer.classList.remove('hidden');
            } catch (error) {
                console.error('Error processing query:', error);
                queryResultMessage.innerText = 'An error occurred while processing the query.';
                imagesContainer.innerHTML = '';
            } finally {
                processButton.disabled = false;
                processButtonText.innerHTML = 'Process Query';
            }
        });

        function showMore(description) {
            alert(description); // Display the complete description in an alert
        }
    </script>
</body>
</html>
"""

@app.route('/')
def hello():
    return "Hello from Colab!"

# Define route to serve HTML form
@app.route('/upload', methods=['GET'])
def upload_form():
    return render_template_string(html_template_v8)

# Define upload route
@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files['file']
    file.save(os.path.join("/content", file.filename))  # Save the file in Colab's /content folder
    extract()  # Ensure this function is defined
    return jsonify({"message": f"File {file.filename} uploaded successfully!"})

@app.route('/process-query', methods=['POST'])
def process_query():
    query = request.form['query']  # Get the query from the form
    # Replace this with your actual processing logic
    result = process_pdf(query)  # Ensure this function is defined and returns appropriate data
    return jsonify(result)

# Run the Flask app
if __name__ == "__main__":
    app.run(port=5000)  # Ensure the app runs on port 50

Ngrok tunnel URL: NgrokTunnel: "https://ca40-34-106-28-250.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 11:05:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 11:05:34] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 11:05:39] "GET /upload HTTP/1.1" 200 -


Removing pre-existing images folder...


 Processing the file: --------------------------------- /content/CONTROL_AND_COORDINATION_CL_X.pdf 


Processing page: 1
Processing page: 2
Extracting image from page: 2, saved as: images/CONTROL_AND_COORDINATION_CL_X.pdf_image_1_0_30.jpeg
Processing page: 3
Extracting image from page: 3, saved as: images/CONTROL_AND_COORDINATION_CL_X.pdf_image_2_0_33.jpeg
Processing page: 4
Processing page: 5
Processing page: 6
Extracting image from page: 6, saved as: images/CONTROL_AND_COORDINATION_CL_X.pdf_image_5_0_40.jpeg
Processing page: 7


INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 11:06:05] "POST /upload HTTP/1.1" 200 -



 
 Sleeping for  5  sec before processing the next document to avoid quota issues. You can disable it: "add_sleep_after_document = False"  


 --- Completed processing. ---
Citation 1: Matched text: 

score:  0.62
file_name:  CONTROL_AND_COORDINATION_CL_X.pdf
page_number:  5
chunk_number:  3
chunk_text:  ing liver glycogen to glucose. 
2. Rise in blood pressure. 
3. Accelerate rate and force of Heartbeat. 
4. Increase in oxygen consumption. 
5. Erection of hair. 
6. Dilation of pupil 
7. Initiates stress response. 
These hormones prepares individual for Fight, Flight & also Frighten, thus also called 3F hormone. 
D) PARATHYROID: Secrete PARATHORMONE:- helps in regulation of blood calcium level if it is somehow decreased. 
E) THYMUS: Secrete THYMOSIN: - helps in differentiation of T-Lymphocytes. PROGESTERON & ESTROGEN:- stimulate 
Uterine lining for embryo implantation to maintain pregnancy, prepare mammary gland for lactation and inhibits 
Ovulation. 
1. RELAXIN: relax pubic symphysis

INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 11:06:59] "POST /process-query HTTP/1.1" 200 -


Hormones are biogenetic organic materials of low molecular weight secreted directly into the blood in very little amounts, showing distant effects acting indirectly through either cell organelles or the enzyme system. 



In [ ]:
!pip install pyngrok flask

from flask import Flask, request, jsonify ,render_template_string
from pyngrok import ngrok
import os

# Initialize Flask app
app = Flask(__name__)

# Start ngrok tunnel
public_url = ngrok.connect(addr='5000') # Changed 'port' to 'addr'
print("Ngrok tunnel URL:", public_url)

# Define the HTML template

html_template_v8="""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Multimodal RAG System</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css">
    <style>
        .loading {
            animation: spin 1s linear infinite;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .scroll-area {
            max-height: 300px;
            overflow-y: auto;
        }
        .image-container {
            margin-bottom: 1.5rem; /* Margin between each image and description */
        }
        .image-description {
            margin-top: 0.5rem; /* Margin between image and description */
            text-align: center; /* Center the description below the image */
            color: #4B5563; /* Gray color for the description text */
        }
        .gallery-overlay {
            position: fixed;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background-color: rgba(0, 0, 0, 0.8);
            z-index: 50;
            display: none;
            justify-content: center;
            align-items: center;
            padding: 1rem;
        }
        .gallery-img {
            max-height: 70vh;
            object-fit: contain;
            border-radius: 0.5rem; /* Rounded corners */
        }
    </style>
</head>
<body class="bg-gray-100">
    <div class="container mx-auto p-4 max-w-4xl">
        <h1 class="text-3xl font-bold mb-8 text-center">Multimodal RAG System</h1>

        <div class="bg-white shadow-md rounded-lg mb-8">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Upload File</h2>
                <form id="uploadForm" class="space-y-4">
                    <input type="file" name="file" id="fileInput" required class="border p-2 rounded w-full" />
                    <button type="submit" id="uploadButton" class="bg-blue-500 text-white p-2 rounded w-full">
                        <span id="uploadButtonText">Upload</span>
                    </button>
                </form>
                <p id="uploadMessage" class="mt-2 text-sm text-gray-600"></p>
            </div>
        </div>

        <div class="bg-white shadow-md rounded-lg mb-8">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Process Query</h2>
                <form id="queryForm" class="space-y-4">
                    <input
                        type="text"
                        name="query"
                        id="queryInput"
                        placeholder="Enter your query"
                        required
                        class="border p-2 rounded w-full"
                    />
                    <button type="submit" id="processButton" class="bg-green-500 text-white p-2 rounded w-full">
                        <span id="processButtonText">Process Query</span>
                    </button>
                </form>
            </div>
        </div>

        <div id="resultContainer" class="hidden bg-white shadow-md rounded-lg">
            <div class="p-4">
                <h2 class="text-xl font-semibold">Query Results</h2>
                <div class="scroll-area rounded-md border p-4">
                    <p id="queryResultText" class="text-gray-700 leading-relaxed"></p>
                </div>
                <h3 class="text-lg font-semibold mb-3">Images and Descriptions</h3>
                <div id="imagesContainer" class="grid grid-cols-2 sm:grid-cols-3 md:grid-cols-3 gap-4"></div>
            </div>
        </div>
    </div>

    <!-- Image Gallery Overlay -->
    <div id="galleryOverlay" class="gallery-overlay">
        <div class="relative w-full max-w-4xl">
            <button id="closeGallery" class="absolute top-2 right-2 text-white z-10">
                <i class="fas fa-times h-6 w-6"></i>
            </button>
            <div class="flex items-center justify-between">
                <button id="prevImage" class="text-white">
                    <i class="fas fa-chevron-left h-8 w-8"></i>
                </button>
                <img id="selectedImage" class="gallery-img" src="" alt="Selected" />
                <button id="nextImage" class="text-white">
                    <i class="fas fa-chevron-right h-8 w-8"></i>
                </button>
            </div>
            <div class="scroll-area mt-4 h-24 rounded-md bg-white/10 p-4">
                <p id="imageDescription" class="text-white text-sm leading-relaxed"></p>
            </div>
        </div>
    </div>

    <script>
        document.getElementById('uploadForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const uploadButton = document.getElementById('uploadButton');
            const uploadButtonText = document.getElementById('uploadButtonText');
            const uploadMessage = document.getElementById('uploadMessage');

            uploadButton.disabled = true;
            uploadButtonText.innerHTML = '<i class="fas fa-spinner fa-spin loading"></i> Uploading...';

            const formData = new FormData(event.target);
            try {
                const response = await fetch('/upload', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                uploadMessage.innerText = result.message;
                document.getElementById('processButton').disabled = false;
            } catch (error) {
                console.error('Error uploading file:', error);
                uploadMessage.innerText = 'Error uploading file';
            } finally {
                uploadButton.disabled = false;
                uploadButtonText.innerHTML = 'Upload';
            }
        });

        document.getElementById('queryForm').addEventListener('submit', async function(event) {
            event.preventDefault();
            const processButton = document.getElementById('processButton');
            const processButtonText = document.getElementById('processButtonText');
            const queryResultMessage = document.getElementById('queryResultMessage');
            const imagesContainer = document.getElementById('imagesContainer');
            const resultContainer = document.getElementById('resultContainer');
            const queryResultText = document.getElementById('queryResultText');

            processButton.disabled = true;
            processButtonText.innerHTML = '<i class="fas fa-spinner fa-spin loading"></i> Processing...';

            const formData = new FormData(event.target);
            try {
                const response = await fetch('/process-query', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                if (result.error) {
                    queryResultMessage.innerText = result.error;
                    imagesContainer.innerHTML = '';
                } else {
                    queryResultMessage.innerText = result.result;
                    queryResultText.innerText = result.result; // Set text result
                    imagesContainer.innerHTML = '';
                    result.images_and_descriptions.forEach((image, index) => {
                        const imageDiv = document.createElement('div');
                        imageDiv.className = 'image-container';
                        imageDiv.innerHTML = `
                            <img src="${image.img_url}" alt="Thumbnail ${index + 1}" class="max-w-full h-auto rounded-lg shadow-md cursor-pointer" onclick="openImage(${index})" />
                            <p class="image-description">${image.description}</p>
                        `;
                        imagesContainer.appendChild(imageDiv);
                    });
                }
                resultContainer.classList.remove('hidden');
            } catch (error) {
                console.error('Error processing query:', error);
                queryResultMessage.innerText = 'An error occurred while processing the query.';
                imagesContainer.innerHTML = '';
            } finally {
                processButton.disabled = false;
                processButtonText.innerHTML = 'Process Query';
            }
        });

        // Image gallery functions
        let selectedIndex = null;
        const images = []; // Store images globally

        function openImage(index) {
            selectedIndex = index;
            document.getElementById('galleryOverlay').style.display = 'flex';
            displayImage();
        }

        function closeImage() {
            document.getElementById('galleryOverlay').style.display = 'none';
        }

        function nextImage() {
            selectedIndex = (selectedIndex === null || selectedIndex === images.length - 1) ? 0 : selectedIndex + 1;
            displayImage();
        }

        function prevImage() {
            selectedIndex = (selectedIndex === null || selectedIndex === 0) ? images.length - 1 : selectedIndex - 1;
            displayImage();
        }

        function displayImage() {
            const selectedImage = document.getElementById('selectedImage');
            const imageDescription = document.getElementById('imageDescription');
            selectedImage.src = images[selectedIndex].img_url;
            imageDescription.innerText = images[selectedIndex].description;
        }

        // Attach click events for gallery navigation
        document.getElementById('closeGallery').addEventListener('click', closeImage);
        document.getElementById('nextImage').addEventListener('click', nextImage);
        document.getElementById('prevImage').addEventListener('click', prevImage);
    </script>
</body>
</html>

"""

@app.route('/')
def hello():
    return "Hello from Colab!"

# Define route to serve HTML form
@app.route('/upload', methods=['GET'])
def upload_form():
    return render_template_string(html_template_v8)

# Define upload route
@app.route('/upload', methods=['POST'])
def upload_file():
    file = request.files['file']
    file.save(os.path.join("/content", file.filename))  # Save the file in Colab's /content folder
    extract()
    return jsonify({"message": f"File {file.filename} uploaded successfully!"})

@app.route('/process-query', methods=['POST'])
def process_query():
    query = request.form['query']  # Get the query from the form
    # Replace this with your processing function
    result = process_pdf(query)  # Assume you have a function that processes the query
    return jsonify(result)


# text_metadata_df, image_metadata_df, matching_results_text, matching_result_images


# Define route to display images
@app.route('/images')
def display_images_route():
    return render_template_string(imagetemplate, matched_images=images_and_descriptions)


# Run the Flask app
if __name__ == "__main__":
    app.run()


Ngrok tunnel URL: NgrokTunnel: "https://2bf8-34-106-28-250.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 10:42:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 10:42:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 10:42:13] "GET /upload HTTP/1.1" 200 -


In [ ]:
# print(matching_results_image)


bucket_name='test_1_0'
images_and_descriptions = []
for key, image in matching_results_image.items(): # Use .items() to iterate through key-value pairs
 if not isinstance(image, dict):
    print(f"Item is not a dictionary: {image}")
 elif "img_path" not in image:
    print(f"'img_path' key missing: {image}")
 else: # Access 'img_path' only if it's a dictionary and contains the key
    local_image_path = image["img_path"]
    gcs_image_path = f"images/{os.path.basename(local_image_path)}"
    img_url = upload_to_gcs(local_image_path, bucket_name, gcs_image_path)  # Upload to GCS
    images_and_descriptions.append({
                "img_url": img_url,
                "description": image["image_description"]
            })
print(images_and_descriptions)

Uniform bucket-level access was disabled for test_1_0.
Uniform bucket-level access was disabled for test_1_0.
Uniform bucket-level access was disabled for test_1_0.
[{'img_url': 'https://storage.googleapis.com/test_1_0/images/CONTROL_AND_COORDINATION_CL_X.pdf_image_5_0_40.jpeg', 'description': 'The illustration depicts the location of endocrine glands in the human body, comparing male (a) and female (b) anatomy.  Shared glands like the pineal gland, pituitary gland, thyroid, parathyroid glands, thymus, pancreas, and adrenal glands are shown in the same relative positions in both figures. The male figure highlights the testes, while the female figure highlights the ovaries. \n'}, {'img_url': 'https://storage.googleapis.com/test_1_0/images/CONTROL_AND_COORDINATION_CL_X.pdf_image_2_0_33.jpeg', 'description': 'The diagram illustrates the pathway of a reflex arc, a rapid and involuntary response to a stimulus. In this instance, heat or pain receptors in the skin detect a stimulus (e.g., tou